# AI Chatbot Training Visualization

This notebook provides an interactive way to visualize the training process of the AI Chatbot, including:

- Data exploration and preprocessing
- Model architecture visualization
- Training progress monitoring
- Performance metrics analysis
- Interactive testing of the trained model

## Table of Contents
1. [Setup and Imports](#setup)
2. [Data Exploration](#data-exploration)
3. [Model Training](#model-training)
4. [Training Visualization](#training-visualization)
5. [Model Testing](#model-testing)
6. [Performance Analysis](#performance-analysis)


## 1. Setup and Imports {#setup}


In [ ]:
# Import required libraries
import os
import sys
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Add the src directory to the path
sys.path.append('../src')

# Import our custom utilities
from nltk_utils import load_intents, preprocess_data, create_training_data
from train import create_model, train_model, plot_training_history, evaluate_model

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure TensorFlow for better performance
tf.config.optimizer.set_jit(True)

print("✅ All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Python version: {sys.version}")


## 2. Data Exploration {#data-exploration}


In [ ]:
# Load and explore the intents data
intents_data = load_intents('../data/intents.json')

if intents_data:
    print("📊 Intents Data Overview:")
    print(f"Number of intents: {len(intents_data['intents'])}")
    
    # Create a summary DataFrame
    intent_summary = []
    for intent in intents_data['intents']:
        intent_summary.append({
            'Intent': intent['tag'],
            'Patterns': len(intent['patterns']),
            'Responses': len(intent['responses']),
            'Sample Pattern': intent['patterns'][0] if intent['patterns'] else '',
            'Sample Response': intent['responses'][0] if intent['responses'] else ''
        })
    
    df_intents = pd.DataFrame(intent_summary)
    print("\n📋 Intent Summary:")
    display(df_intents)
    
    # Visualize intent distribution
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Patterns per intent
    ax1.bar(df_intents['Intent'], df_intents['Patterns'], color='skyblue', alpha=0.7)
    ax1.set_title('Number of Patterns per Intent', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Intent')
    ax1.set_ylabel('Number of Patterns')
    ax1.tick_params(axis='x', rotation=45)
    
    # Responses per intent
    ax2.bar(df_intents['Intent'], df_intents['Responses'], color='lightcoral', alpha=0.7)
    ax2.set_title('Number of Responses per Intent', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Intent')
    ax2.set_ylabel('Number of Responses')
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Print total statistics
    total_patterns = df_intents['Patterns'].sum()
    total_responses = df_intents['Responses'].sum()
    print(f"\n📈 Total Statistics:")
    print(f"Total patterns: {total_patterns}")
    print(f"Total responses: {total_responses}")
    print(f"Average patterns per intent: {total_patterns / len(intents_data['intents']):.1f}")
    print(f"Average responses per intent: {total_responses / len(intents_data['intents']):.1f}")
else:
    print("❌ Failed to load intents data")


In [ ]:
# Preprocess the data and explore vocabulary
print("🔧 Preprocessing data...")
words, labels, xy = preprocess_data(intents_data)

# Create training data
print("📊 Creating training data...")
X, y = create_training_data(words, labels, xy)

print(f"\n📈 Data Statistics:")
print(f"Vocabulary size: {len(words)}")
print(f"Number of labels: {len(labels)}")
print(f"Total training samples: {X.shape[0]}")
print(f"Feature vector size: {X.shape[1]}")

# Visualize vocabulary distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Word length distribution
word_lengths = [len(word) for word in words]
ax1.hist(word_lengths, bins=20, color='skyblue', alpha=0.7, edgecolor='black')
ax1.set_title('Distribution of Word Lengths in Vocabulary', fontsize=14, fontweight='bold')
ax1.set_xlabel('Word Length')
ax1.set_ylabel('Frequency')
ax1.grid(True, alpha=0.3)

# Label distribution
label_counts = np.bincount(y)
ax2.bar(labels, label_counts, color='lightcoral', alpha=0.7)
ax2.set_title('Distribution of Training Samples per Intent', fontsize=14, fontweight='bold')
ax2.set_xlabel('Intent')
ax2.set_ylabel('Number of Samples')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show sample of vocabulary
print(f"\n📝 Sample Vocabulary (first 20 words):")
print(words[:20])

# Show label distribution
print(f"\n🏷️  Label Distribution:")
for i, label in enumerate(labels):
    count = np.sum(y == i)
    print(f"  {label}: {count} samples")


## 3. Model Training {#model-training}


In [ ]:
# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"📊 Data Split:")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Create the model
print(f"\n🧠 Creating neural network model...")
model = create_model(X_train.shape[1], len(labels))

# Display model architecture
print(f"\n📋 Model Architecture:")
model.summary()

# Visualize model architecture
keras.utils.plot_model(model, to_file='../models/model_architecture.png', 
                      show_shapes=True, show_layer_names=True, rankdir='TB')
print("✅ Model architecture saved to models/model_architecture.png")


In [ ]:
# Train the model
print("🚀 Starting model training...")
history = train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32)

# Save the trained model
model.save('../models/chatbot_model.h5')
print("✅ Model saved to models/chatbot_model.h5")

# Save words and labels
import pickle
with open('../models/words.pkl', 'wb') as f:
    pickle.dump(words, f)
with open('../models/labels.pkl', 'wb') as f:
    pickle.dump(labels, f)
print("✅ Vocabulary data saved")


## 4. Training Visualization {#training-visualization}


In [ ]:
# Create comprehensive training visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Training and validation accuracy
axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
axes[0, 0].set_title('Model Accuracy Over Time', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Training and validation loss
axes[0, 1].plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
axes[0, 1].plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
axes[0, 1].set_title('Model Loss Over Time', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Learning rate (if available)
if 'lr' in history.history:
    axes[1, 0].plot(history.history['lr'], color='green', linewidth=2)
    axes[1, 0].set_title('Learning Rate Over Time', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].grid(True, alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'Learning Rate Not Available', 
                    ha='center', va='center', transform=axes[1, 0].transAxes)
    axes[1, 0].set_title('Learning Rate Over Time', fontsize=14, fontweight='bold')

# Final metrics comparison
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

metrics = ['Training', 'Validation']
accuracies = [final_train_acc, final_val_acc]
losses = [final_train_loss, final_val_loss]

x = np.arange(len(metrics))
width = 0.35

axes[1, 1].bar(x - width/2, accuracies, width, label='Accuracy', color='skyblue', alpha=0.7)
axes[1, 1].bar(x + width/2, losses, width, label='Loss', color='lightcoral', alpha=0.7)
axes[1, 1].set_title('Final Training vs Validation Metrics', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Dataset')
axes[1, 1].set_ylabel('Value')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(metrics)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print training summary
print("📊 Training Summary:")
print(f"Final Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")
print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")
print(f"Total Epochs: {len(history.history['accuracy'])}")

# Check for overfitting
if final_val_acc < final_train_acc - 0.1:
    print("⚠️  Warning: Potential overfitting detected (validation accuracy significantly lower than training)")
elif final_val_acc > final_train_acc:
    print("✅ Good generalization: Validation accuracy is higher than training accuracy")
else:
    print("✅ Balanced training: Training and validation metrics are close")


## 5. Model Testing {#model-testing}


In [ ]:
# Test the model on test data
print("🧪 Testing model on test data...")

# Get predictions
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# Calculate accuracy
test_accuracy = np.mean(predicted_classes == y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Create confusion matrix
cm = confusion_matrix(y_test, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.show()

# Print classification report
print("\n📊 Classification Report:")
print(classification_report(y_test, predicted_classes, target_names=labels))

# Test with sample inputs
print("\n🤖 Interactive Testing:")
test_messages = [
    "Hello!",
    "How are you?",
    "What's your name?",
    "Thank you very much",
    "Goodbye!",
    "Can you help me?",
    "What's the weather like?",
    "I don't understand this at all"
]

for message in test_messages:
    # This would require importing the chatbot class
    # For now, we'll just show the message
    print(f"Test message: '{message}'")
